### authorizing the web app

In [2]:
from flask import Flask, request
import webbrowser
import os
import signal

def get_spotify_auth_code():
    app = Flask(__name__)
    
    client_id = "REDACTED"
    client_secret = "REDACTED"
    redirect_uri = "http://127.0.0.1:8889/callback"
    
    auth_code = None
    
    @app.route('/callback')
    def callback():
        nonlocal auth_code
        auth_code = request.args.get('code')
        print(f"\nAuthorization code: {auth_code}\n")
        os.kill(os.getpid(), signal.SIGINT)
        return "Authorization successful! You can close this window."
    
    scope = "user-read-currently-playing user-read-playback-state user-top-read user-read-recently-played user-library-read user-read-private"
    auth_url = f"https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&scope={scope}"
    
    webbrowser.open(auth_url)
    
    try:
        app.run(port=8889)
    except KeyboardInterrupt:
        pass
    
    return auth_code

# Run it
auth_code = get_spotify_auth_code()
print(f"Got code: {auth_code}")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8889
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2026 10:29:37] "GET /callback?code=AQBq97FYL1eXlJzC_MJJWaFW6HYeSFfxdEP_TX4vlYi6pe0YQ9TaQAzBM5r3x5Xu4jGkQ5qt8lmseSKMNEaHSZegPG1hn8lQECb521P9On9X0BSiHNimLO_uQvUjpg86vBUymB_guHV_ILFhgOkix9JevqEg-MZylmum0W3DUnPzcxRThAV8HV-R9dcTH5o4kECW3Rqth5Ppl_NreIp6pg8EwUUbhT5KUEt-UQZum3jCEHDzUJYNVvy5dGz6FXncfGZFPR9XZ6-9yV_BM_1sW9kiqjatMQlzKIt8NLfdIITTxuNR85q8nL1e59DbGaKSY7AUrHO-Ik_wepHPCTTVVfc-b2RsXBIUNw HTTP/1.1" 200 -



Authorization code: AQBq97FYL1eXlJzC_MJJWaFW6HYeSFfxdEP_TX4vlYi6pe0YQ9TaQAzBM5r3x5Xu4jGkQ5qt8lmseSKMNEaHSZegPG1hn8lQECb521P9On9X0BSiHNimLO_uQvUjpg86vBUymB_guHV_ILFhgOkix9JevqEg-MZylmum0W3DUnPzcxRThAV8HV-R9dcTH5o4kECW3Rqth5Ppl_NreIp6pg8EwUUbhT5KUEt-UQZum3jCEHDzUJYNVvy5dGz6FXncfGZFPR9XZ6-9yV_BM_1sW9kiqjatMQlzKIt8NLfdIITTxuNR85q8nL1e59DbGaKSY7AUrHO-Ik_wepHPCTTVVfc-b2RsXBIUNw

Got code: AQBq97FYL1eXlJzC_MJJWaFW6HYeSFfxdEP_TX4vlYi6pe0YQ9TaQAzBM5r3x5Xu4jGkQ5qt8lmseSKMNEaHSZegPG1hn8lQECb521P9On9X0BSiHNimLO_uQvUjpg86vBUymB_guHV_ILFhgOkix9JevqEg-MZylmum0W3DUnPzcxRThAV8HV-R9dcTH5o4kECW3Rqth5Ppl_NreIp6pg8EwUUbhT5KUEt-UQZum3jCEHDzUJYNVvy5dGz6FXncfGZFPR9XZ6-9yV_BM_1sW9kiqjatMQlzKIt8NLfdIITTxuNR85q8nL1e59DbGaKSY7AUrHO-Ik_wepHPCTTVVfc-b2RsXBIUNw


### authorized! now getting access and refresh tokens

In [3]:
import requests
import base64


client_id = "REDACTED"
client_secret = "REDACTED"
code = auth_code
# Encode credentials
auth_str = f"{client_id}:{client_secret}"
auth_bytes = auth_str.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# exchanging code for token
url = "https://accounts.spotify.com/api/token"
headers = {"Authorization": f"Basic {auth_base64}"}
data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://127.0.0.1:8889/callback"
}

response = requests.post(url, headers=headers, data=data)
token_data = response.json()

print(token_data)

access_token = token_data['access_token']
refresh_token = token_data['refresh_token']

print(f"Access token: {access_token}")
print(f"Refresh token: {refresh_token}")

{'access_token': 'BQDiz_wevH9nUbX_3Kj0NhtWOH_dNqWddKXieknT8BYrBVjQugq_gmPDqx7rcvtTcrcY3SfLep062rBu9KyE-e0AIgr3Y5nkbK4HIRfAoy5rqxSINM_xs8LfNwynn0UxIjJhf4luz2McK7-uNErvXU2vc7QqXva7I8nibHDAxOx3Uhv1RpCrA-0tmKPuukpTxL4ZjrHdgVoV6U1odgfui4g-ZchzIrKJeVHm9LpAogJuCoxJfkkDsGDjfeo7vqvpiwwpaGPL8cSVFFd5xgC7JqwyYsGJ', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQB0luK3PIn2019xaBKnyRFno84LXIHlk6O2NiPwt2hOdZDovmLOlAuaUR6fHcYDLlMa8jssXYKMBRGS4ZBf4sbKPDZeobPovrFca1NGNs8w9eJYgjIH7ULj1XAH7SFjYvk', 'scope': 'user-library-read user-read-playback-state user-read-currently-playing user-read-recently-played user-top-read user-read-private'}
Access token: BQDiz_wevH9nUbX_3Kj0NhtWOH_dNqWddKXieknT8BYrBVjQugq_gmPDqx7rcvtTcrcY3SfLep062rBu9KyE-e0AIgr3Y5nkbK4HIRfAoy5rqxSINM_xs8LfNwynn0UxIjJhf4luz2McK7-uNErvXU2vc7QqXva7I8nibHDAxOx3Uhv1RpCrA-0tmKPuukpTxL4ZjrHdgVoV6U1odgfui4g-ZchzIrKJeVHm9LpAogJuCoxJfkkDsGDjfeo7vqvpiwwpaGPL8cSVFFd5xgC7JqwyYsGJ
Refresh token: AQB0luK3PIn2019xaBKnyRFno84LXIHlk6O2NiPwt2hO

### tokens received!

### uh oh, this access token only valid for one hour! what do i do? 🤔

In [4]:
import requests
import base64
import json
import time
from datetime import datetime, timedelta

class SpotifyAuth:
    def __init__(self, client_id, client_secret, tokens_file='spotify_tokens.json'):
        self.client_id = client_id
        self.client_secret = client_secret
        self.tokens_file = tokens_file
        self.load_tokens()
    
    def load_tokens(self):
        """Load tokens from file"""
        with open(self.tokens_file, 'r') as f:
            data = json.load(f)
            self.access_token = data['access_token']
            self.refresh_token = data['refresh_token']
            self.expires_at = data.get('expires_at', time.time())
    
    def save_tokens(self):
        """Save tokens to file"""
        data = {
            'access_token': self.access_token,
            'refresh_token': self.refresh_token,
            'expires_at': self.expires_at
        }
        with open(self.tokens_file, 'w') as f:
            json.dump(data, f)
    
    def is_expired(self):
        """Check if access token is expired"""
        return time.time() >= self.expires_at
    
    def refresh_access_token(self):
        """Get a new access token using refresh token"""
        auth_str = f"{self.client_id}:{self.client_secret}"
        auth_base64 = base64.b64encode(auth_str.encode()).decode()
        
        url = "https://accounts.spotify.com/api/token"
        headers = {"Authorization": f"Basic {auth_base64}"}
        data = {
            "grant_type": "refresh_token",
            "refresh_token": self.refresh_token
        }
        
        response = requests.post(url, headers=headers, data=data)
        tokens = response.json()
        
        self.access_token = tokens['access_token']
        # Token expires in 3600 seconds (1 hour), refresh 5 min early
        self.expires_at = time.time() + tokens['expires_in'] - 300
        self.save_tokens()
    
    def get_valid_token(self):
        """Get a valid access token, refreshing if needed"""
        if self.is_expired():
            print("Token expired, refreshing...")
            self.refresh_access_token()
        return self.access_token

spotify_auth = SpotifyAuth(client_id, client_secret)
spotify_auth.access_token = access_token
spotify_auth.refresh_token = refresh_token
spotify_auth.expires_at = time.time() + 3600  # 1 hour from current
spotify_auth.save_tokens()
print("tokens file created")


tokens = {
    'access_token': 'BQAa8Ns49noTNzLYlP_IqASVzBfGekui0aJhQnIU-4nx1gs4wpwzeFfrKUJocLC5lbTqi98N4Q2NmNlo4SY4gGVRvc0KoazCs0QAb_Bcs0llGuQJluo4nh6Uxy-YpUV8b8I8Kj_9at0GSFXSQKwiLsBJoHE8DHu1JlDdsQI62fD3zCxBQ4uDwpH1zEnY2MSXLM__X06L38dlJqDLx58BX70vNChxCeA70M8eHZN7QC_YDFqOv49qn5iH4pSAhkR229i13Noa7dHzs3t3Xa-93xT4_DT5',
    'refresh_token': 'AQCybSVcUnp9moxd2qA9XpCXjIvsysRXO8UlTfMb1VMh8rN0nHa3vL-sbkk1eMfVdVXkaptBQau3q4ofKxCgWUw9jp4vzU-HlSuFA95dSrh_EAu3j8HyH_OHu4wvWkE1VG4',
    'expires_at': 0
}

with open('spotify_tokens.json', 'w') as f:
    json.dump(tokens, f)

print("tokens file created")


# Usage
auth = SpotifyAuth(
    client_id='REDACTED',
    client_secret='REDACTED'
)

FileNotFoundError: [Errno 2] No such file or directory: 'spotify_tokens.json'

### Free For All You Want To Do (limited by spotify's web API)

### currently playing track, with auto refreshing access token!

In [ ]:
token = spotify_auth.get_valid_token()
url = "https://api.spotify.com/v1/me/player/currently-playing"
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    track_name = data['item']['name']
    artist_name = data['item']['artists'][0]['name']
    progress_ms = data['progress_ms'] # current duration in milliseconds
    duration_ms = data['item']['duration_ms'] # total song length

    progress_sec = progress_ms // 1000
    duration_sec = duration_ms // 1000
    
    progress_min = progress_sec // 60
    progress_s = progress_sec % 60
    
    duration_min = duration_sec // 60
    duration_s = duration_sec % 60
    
    print(f"Currently playing: {track_name} by {artist_name}")
    print(f"Progress: {progress_min}:{progress_s:02d} / {duration_min}:{duration_s:02d}")
elif response.status_code == 204:

    print("No track currently playing")
    url = "https://api.spotify.com/v1/me/player/recently-played?limit=1"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        last_track = data['items'][0]['track']
        track_name = last_track['name']
        artist_name = last_track['artists'][0]['name']
        print(f"Last played: {track_name} by {artist_name}")
    else:
        print("Could not get recently played")
else:
    print(f"Error: {response.status_code}")

NameError: name 'spotify_auth' is not defined

### current spotify queue!

In [ ]:
# Get queue
url = "https://api.spotify.com/v1/me/player/queue"
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    
    # Currently playing
    current = data['currently_playing']
    print(f"Currently playing: {current['name']} by {current['artists'][0]['name']}\n")
    
    # Queue
    print("Queue:")
    for i, track in enumerate(data['queue'], 1):
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        print(f"{i}. {track_name} by {artist_name}")
elif response.status_code == 204:
    print("There are no songs in the queue")
else:
    print(f"Error: {response.status_code}")
    print(response.json())

Currently playing: Losalamitoslatinfunklovesong by Gene Harris

Queue:
1. Crying Laughing Loving Lying (From "The Holdovers") by Labi Siffre
2. Tema di Andrea by Alberto Baldan Bembo
3. Bless the Telephone by Labi Siffre
4. Home At Last by Steely Dan
5. Can We Pretend by Bill Withers
6. タイム・リミット by CASIOPEA
7. All I Wanna Do - Remastered 2009 by The Beach Boys
8. If I Should Die Tonight by Marvin Gaye
9. Tudo O Que Você Podia Ser by Milton Nascimento
10. We Have Love by Amnesty
11. (They Long To Be) Close To You by Carpenters
12. ファー・アウェイ by CASIOPEA
13. She’s My Baby - 2014 Remaster by Wings
14. Tomorrow Is Gone by Jode
15. Let Me Prove My Love to You by The Main Ingredient
16. Paper Machine by 藤丸バンド
17. Hold On - Remastered 2010 by John Lennon
18. Oba, Lá Vem Ela by Jorge Ben Jor
19. Where Are We Going? by Marvin Gaye
20. ブリーズ by 稲垣次郎とソウル・メディア


### top artists, followed by top songs of all time

In [ ]:
url = "https://api.spotify.com/v1/me/top/artists?time_range=long_term&limit=50"
headers = {"Authorization": f"Bearer {token}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Top Artists (All Time):")
    for i, artist in enumerate(data['items'], 1):
        print(f"{i}. {artist['name']}")
else:
    print("Error occurred")

Top Artists (All Time):
1. Stereolab
2. Radiohead
3. Lamp
4. Kendrick Lamar
5. Kanye West
6. Masayoshi Takanaka
7. Fiona Apple
8. Tyler, The Creator
9. Nujabes
10. Mac DeMarco
11. Frank Ocean
12. naran ratan
13. The Beatles
14. Uyama Hiroto
15. Mort Garson
16. Childish Gambino
17. Joji
18. Dean Blunt
19. Steve Lacy
20. Laufey
21. Mitski
22. Yves Tumor
23. Cocteau Twins
24. Yeat
25. CASIOPEA
26. Have A Nice Life
27. Travis Scott
28. Taeko Onuki
29. Future
30. Clairo
31. Daniel Caesar
32. Playboi Carti
33. glass beach
34. Westside Gunn
35. Lil Yachty
36. Earl Sweatshirt
37. Baby Keem
38. Panchiko
39. Ken Carson
40. Frank Sinatra
41. Shigeru Suzuki
42. Macabre Plaza
43. Drake
44. Tame Impala
45. Ichiko Aoba
46. Ryo Fukui
47. Bruno Mars
48. Geese
49. 間宮貴子
50. Cortex


In [ ]:
url = "https://api.spotify.com/v1/me/top/tracks?time_range=long_term&limit=50"
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    
    print("\nTop Tracks (All Time):")
    for i, track in enumerate(data['items'], 1):
        artist = track['artists'][0]['name']
        print(f"{i}. {track['name']} by {artist}")


Top Tracks (All Time):
1. Solo by Frank Ocean
2. Seigfried by Frank Ocean
3. The First Taste by Fiona Apple
4. Miss Misery by Elliott Smith
5. Sullen Girl by Fiona Apple
6. I Know by Fiona Apple
7. Paper Bag by Fiona Apple
8. I Want You To Love Me by Fiona Apple
9. Plantasia by Mort Garson
10. Carrion by Fiona Apple
11. Fast As You Can by Fiona Apple
12. Criminal by Fiona Apple
13. Skyline To by Frank Ocean
14. The Way Things Are by Fiona Apple
15. The Percocet & Stripper Joint by Future
16. Sleep to Dream by Fiona Apple
17. Get Gone by Fiona Apple
18. Under The Table by Fiona Apple
19. Extraordinary Machine by Fiona Apple
20. Oh! Darling - Remastered 2009 by The Beatles
21. Nude by Radiohead
22. I Want You by Mitski
23. Valentine by Fiona Apple
24. Exit Music (For A Film) by Radiohead
25. God Is by Kanye West
26. Sur Niragas Ho by Anandi Joshi
27. A Mistake by Fiona Apple
28. Self Control by Frank Ocean
29. We Major by Kanye West
30. Like Him (feat. Lola Young) by Tyler, The Creator
